# 🎓 Capstone Project: End-to-End LLM Deployment

**Module 05 | Capstone**

In this capstone, you'll apply everything you've learned to build a complete, production-ready NLP system.

## Project Overview

You will build a **Multi-Task NLP Service** that:
1. Fine-tunes a model for your chosen task
2. Optimizes the model for deployment
3. Creates a REST API and interactive UI
4. Containerizes the application

---

## Learning Objectives

By completing this capstone, you will demonstrate:
- [ ] Fine-tuning transformer models
- [ ] Applying optimization techniques
- [ ] Building production APIs
- [ ] Creating user interfaces
- [ ] Containerization skills

---

In [ ]:
%%capture
!pip install transformers datasets evaluate torch accelerate gradio fastapi uvicorn

In [ ]:
import torch
import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
print("\n🎓 Let's build your capstone project!")

---

## 📋 Project Requirements

### Minimum Requirements

| Component | Requirement |
|-----------|-------------|
| **Model** | Fine-tuned transformer model |
| **Optimization** | Apply at least one optimization technique |
| **API** | FastAPI with /predict endpoint |
| **UI** | Gradio interface |
| **Docker** | Working Dockerfile |

### Suggested Tasks (Choose One)

1. **Sentiment Analysis** - Movie/Product reviews
2. **Text Classification** - News/Topic categorization
3. **Summarization** - Article summarization
4. **Question Answering** - Extractive QA
5. **Custom Task** - Propose your own!

---

## Phase 1: Data Preparation & Fine-Tuning

### Step 1.1: Load and Explore Your Dataset

In [ ]:
from datasets import load_dataset

# Example: Loading a sentiment analysis dataset
# TODO: Replace with your chosen dataset
dataset = load_dataset("imdb")

print("Dataset overview:")
print(f"  Train: {len(dataset['train'])} samples")
print(f"  Test: {len(dataset['test'])} samples")

# Explore
print("\nSample:")
print(f"  Text: {dataset['train'][0]['text'][:200]}...")
print(f"  Label: {dataset['train'][0]['label']}")

In [ ]:
# Use subset for faster training (adjust for your resources)
train_size = 5000
test_size = 1000

train_data = dataset['train'].shuffle(seed=42).select(range(train_size))
test_data = dataset['test'].shuffle(seed=42).select(range(test_size))

print(f"Using {train_size} training and {test_size} test samples")

### Step 1.2: Tokenize Data

In [ ]:
from transformers import AutoTokenizer

# TODO: Choose your base model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(
        examples['text'],
        padding='max_length',
        truncation=True,
        max_length=256
    )

train_tokenized = train_data.map(tokenize_function, batched=True)
test_tokenized = test_data.map(tokenize_function, batched=True)

# Set format
train_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_tokenized.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

print("✅ Tokenization complete!")

### Step 1.3: Fine-Tune the Model

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import evaluate
import numpy as np

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2
)

# Metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {
        'accuracy': accuracy_metric.compute(predictions=predictions, references=labels)['accuracy'],
        'f1': f1_metric.compute(predictions=predictions, references=labels)['f1']
    }

print(f"Model loaded: {model_name}")
print(f"Parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
# Training configuration
training_args = TrainingArguments(
    output_dir="./capstone_model",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    learning_rate=2e-5,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=test_tokenized,
    compute_metrics=compute_metrics
)

print("Ready to train!")

In [ ]:
# Train!
print("🏃 Starting training...")
trainer.train()

# Evaluate
results = trainer.evaluate()
print(f"\n📊 Final Results:")
print(f"  Accuracy: {results['eval_accuracy']:.2%}")
print(f"  F1 Score: {results['eval_f1']:.4f}")

In [ ]:
# Save the fine-tuned model
model_path = "./capstone_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"\n✅ Model saved to {model_path}")

---

## Phase 2: Model Optimization

Apply at least one optimization technique.

In [ ]:
from copy import deepcopy
import time

# Reload the model
model_optimized = AutoModelForSequenceClassification.from_pretrained(model_path)

print("Original model size:")
original_size = sum(p.numel() * p.element_size() for p in model_optimized.parameters()) / 1024**2
print(f"  {original_size:.1f} MB")

In [ ]:
# Option 1: Dynamic Quantization (INT8)
model_quantized = torch.quantization.quantize_dynamic(
    deepcopy(model_optimized),
    {torch.nn.Linear},
    dtype=torch.qint8
)

print("\n✅ Quantization applied!")
quantized_size = sum(p.nelement() * p.element_size() for p in model_quantized.parameters()) / 1024**2
print(f"Quantized model size: {quantized_size:.1f} MB")
print(f"Size reduction: {(1 - quantized_size/original_size)*100:.1f}%")

In [ ]:
# Benchmark latency
test_text = "This product exceeded all my expectations!"
inputs = tokenizer(test_text, return_tensors="pt")

def benchmark(model, inputs, n=50):
    model.eval()
    # Warmup
    with torch.no_grad():
        for _ in range(5):
            _ = model(**inputs)
    # Benchmark
    start = time.time()
    for _ in range(n):
        with torch.no_grad():
            _ = model(**inputs)
    return (time.time() - start) / n * 1000

original_latency = benchmark(model_optimized, inputs)
quantized_latency = benchmark(model_quantized, inputs)

print(f"\n⏱️ Latency Comparison (CPU):")
print(f"  Original: {original_latency:.2f} ms")
print(f"  Quantized: {quantized_latency:.2f} ms")
print(f"  Speedup: {original_latency/quantized_latency:.2f}x")

In [ ]:
# Use the optimized model for deployment
deployed_model = model_quantized  # or model_optimized if you prefer
print("\n✅ Optimization complete! Using quantized model for deployment.")

---

## Phase 3: Build the API & UI

### Step 3.1: Prediction Function

In [ ]:
import torch.nn.functional as F

# Set up prediction
deployed_model.eval()
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

def predict(text: str) -> dict:
    """Run inference on input text."""
    inputs = tokenizer(
        text,
        return_tensors="pt",
        truncation=True,
        max_length=256
    )
    
    with torch.no_grad():
        outputs = deployed_model(**inputs)
    
    probs = F.softmax(outputs.logits, dim=-1)[0]
    pred_idx = probs.argmax().item()
    
    return {
        "label": id2label[pred_idx],
        "confidence": probs[pred_idx].item(),
        "probabilities": {
            id2label[i]: probs[i].item() 
            for i in range(len(probs))
        }
    }

# Test
print("Testing prediction function:")
print(predict("This movie was absolutely terrible!"))
print(predict("I loved every minute of this experience!"))

### Step 3.2: FastAPI Application

In [ ]:
# Generate FastAPI app code
api_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, Field
from typing import Dict
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Initialize app
app = FastAPI(
    title="Capstone NLP API",
    description="Production-ready sentiment analysis API",
    version="1.0.0"
)

# Load model
MODEL_PATH = "./capstone_finetuned"
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model = torch.quantization.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)
model.eval()

id2label = {0: "NEGATIVE", 1: "POSITIVE"}

# Schemas
class PredictRequest(BaseModel):
    text: str = Field(..., min_length=1, max_length=5000)

class PredictResponse(BaseModel):
    label: str
    confidence: float
    probabilities: Dict[str, float]

# Endpoints
@app.get("/health")
def health():
    return {"status": "healthy", "model": MODEL_PATH}

@app.post("/predict", response_model=PredictResponse)
def predict(request: PredictRequest):
    try:
        inputs = tokenizer(request.text, return_tensors="pt", truncation=True, max_length=256)
        with torch.no_grad():
            outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1)[0]
        pred_idx = probs.argmax().item()
        return {
            "label": id2label[pred_idx],
            "confidence": probs[pred_idx].item(),
            "probabilities": {id2label[i]: probs[i].item() for i in range(len(probs))}
        }
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    import uvicorn
    uvicorn.run(app, host="0.0.0.0", port=8000)
'''

with open("./capstone_api.py", "w") as f:
    f.write(api_code)

print("✅ FastAPI application saved to capstone_api.py")
print("\nRun with: uvicorn capstone_api:app --reload")

### Step 3.3: Gradio Interface

In [ ]:
import gradio as gr

def predict_for_gradio(text):
    if not text.strip():
        return {"Error": 1.0}
    result = predict(text)
    return {
        f"{result['label']} {'😊' if result['label'] == 'POSITIVE' else '😠'}": result['confidence'],
        "Other": 1 - result['confidence']
    }

# Create interface
demo = gr.Interface(
    fn=predict_for_gradio,
    inputs=gr.Textbox(
        label="Enter text for sentiment analysis",
        placeholder="Type your review here...",
        lines=4
    ),
    outputs=gr.Label(label="Sentiment", num_top_classes=2),
    title="🎓 Capstone: Sentiment Analysis",
    description="Fine-tuned and optimized DistilBERT model for sentiment classification.",
    examples=[
        ["This product is amazing! Best purchase I've ever made."],
        ["Terrible quality. Broke after one day."],
        ["It's okay. Nothing special but gets the job done."]
    ],
    theme="soft"
)

demo.launch(share=False, inline=True)

---

## Phase 4: Docker Packaging

In [ ]:
# Generate Dockerfile
dockerfile = '''FROM python:3.10-slim

WORKDIR /app

# Install dependencies
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy model and code
COPY capstone_finetuned/ ./capstone_finetuned/
COPY capstone_api.py .

# Expose port
EXPOSE 8000

# Health check
HEALTHCHECK --interval=30s --timeout=10s --start-period=60s \\
  CMD curl -f http://localhost:8000/health || exit 1

# Run
CMD ["uvicorn", "capstone_api:app", "--host", "0.0.0.0", "--port", "8000"]
'''

requirements = '''transformers==4.35.0
torch==2.1.0
fastapi==0.104.0
uvicorn==0.24.0
pydantic==2.5.0
'''

with open("./Dockerfile", "w") as f:
    f.write(dockerfile)

with open("./requirements.txt", "w") as f:
    f.write(requirements)

print("✅ Docker files created:")
print("  - Dockerfile")
print("  - requirements.txt")
print("\nBuild with: docker build -t capstone-nlp .")
print("Run with: docker run -p 8000:8000 capstone-nlp")

---

## 📋 Submission Checklist

Before submitting, verify:

### Required
- [ ] Fine-tuned model saved to `capstone_finetuned/`
- [ ] Optimization applied (quantization/pruning)
- [ ] FastAPI `capstone_api.py` works
- [ ] Gradio interface runs
- [ ] `Dockerfile` builds successfully

### Evaluation Criteria

| Criterion | Points |
|-----------|--------|
| Model accuracy (>85%) | 20 |
| Optimization benefits documented | 20 |
| API works correctly | 20 |
| UI is functional | 15 |
| Docker builds & runs | 15 |
| Code quality & documentation | 10 |
| **Total** | **100** |

---

## 🎯 Extension Challenges

For extra credit:

1. **Multi-model API** - Add summarization or NER endpoint
2. **Batch processing** - Add `/predict/batch` endpoint
3. **Caching** - Add Redis caching for repeated requests
4. **Monitoring** - Add Prometheus metrics
5. **Cloud deployment** - Deploy to AWS ECS or Hugging Face Spaces

---

## 🎉 Congratulations!

You've completed the LLM Deployment Course! You now have the skills to:

- ✅ Fine-tune transformer models
- ✅ Optimize for production (distillation, pruning, quantization)
- ✅ Build REST APIs with FastAPI
- ✅ Create interactive UIs with Gradio
- ✅ Containerize with Docker
- ✅ Deploy to cloud infrastructure

**Next steps:**
- Build your own NLP applications
- Explore advanced optimization (ONNX, TensorRT)
- Learn about MLOps and CI/CD
- Contribute to open-source ML projects